# This script includes: 

## 1. Import libraries
## 2. Import data
## 3. Data pre-processing
    3.1 Reshape data
    3.2 Split data
## 4. Set up Bayesian optimizer
## 5. CNN model with optimized hyperparameters
    5.1 Run the CNN model
    5.2 Make predictions
    5.3 Classification reports
    5.4 Confusion matrices

## 1. Import libraries

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU

## 2. Import data

In [2]:
# Import data

path = r'C:\Users\ashle\Documents\10-2024 Climate Analysis'

X = pd.read_csv(os.path.join(path, '02 Data Sets', 'X_weather_data_cleaned_preprocessed.csv'),index_col = False) 
y = pd.read_csv(os.path.join(path, '02 Data Sets', 'y_pleasant_cleaned_preprocessed.csv'),index_col = False) 

## 3. Data pre-processing

### 3.1 Reshape data

In [3]:
X

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,14.2,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,14.3,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,14.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,12.4,5,0.82,1.0142,1.13,0.41,3.4,10.7,7.9,13.5


In [7]:
X.shape

(22950, 135)

In [9]:
y

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22946,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22947,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22948,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
y.shape

(22950, 15)

In [13]:
# Convert variables into numpy arrays
X = X.to_numpy()
y = y.to_numpy()

# Split X into 15 groups of 9 types of observations
X = X.reshape(-1, 15, 9)

# Check the shapes
print('X shape: ', X.shape) # should be (22950, 15, 9)
print('y shape: ', y.shape) # should be (22950, 15)

X shape:  (22950, 15, 9)
y shape:  (22950, 15)


### 3.2 Split data

In [15]:
# Split the data into two sets: a training and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 62)

In [17]:
# Check shape of each test and train set
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (16065, 15, 9)
y_train shape:  (16065, 15)
X_test shape:  (6885, 15, 9)
y_test shape:  (6885, 15)


In [19]:
len(X_train[0])

15

In [21]:
len(X_train[0][0])

9

## 4 Set up Bayesian optimizer

In [23]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

'multilabel-indicator'

In [25]:
#Use argmax to get rid of on-hot encoding and supply the numerical value.
y1_train = np.argmax(y_train, axis = 1)
print(y1_train.shape)
y1_train

(16065,)


array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [27]:
type_of_target(y1_train)

'multiclass'

In [29]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

# Define accuracy scorer
score_acc = make_scorer(accuracy_score)

In [33]:
# Set up the parameters
params ={
    'activation': (0, 7),
    'neurons': (10, 100),
    'kernel': (1, 3),
    'optimizer':(0,7),
    'learning_rate': (0.0001, 0.1),
    'batch_size': (16, 128),
    'epochs':(10, 50),
    'layers1':(1,5),
    'layers2':(1,5),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.5)
}

# Activation and optimizer mappings
activation_mapping = {
    0: 'relu',
    1: 'sigmoid',
    2: 'softplus',
    3: 'softsign',
    4: 'tanh',
    5: 'selu',
    6: 'elu',
    7: 'exponential'
}
optimizer_mapping = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

# EarlyStopping
es = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)


# Define Bayesian Optimization Function
def bayesian_optimization_fn(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
                             layers1, layers2, normalization, dropout, dropout_rate):
    activation_func = activation_mapping[round(activation)]
    optimizer_func = optimizer_mapping[round(optimizer)]
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(int(neurons), kernel_size=int(kernel), activation=activation_func, 
                         input_shape=(X_train.shape[1], X_train.shape[2])))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for _ in range(int(layers1)):
            model.add(Dense(int(neurons), activation=activation_func))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate))
        for _ in range(int(layers2)):
            model.add(Dense(int(neurons), activation=activation_func))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(15, activation='softmax'))  # Output layer
        model.compile(optimizer=optimizer_func, 
                      loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])
        return model
         
    nn = KerasClassifier(build_fn=cnn_model, epochs=int(epochs), batch_size=int(batch_size), verbose=0)
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(nn, X_train, y1_train, cv=kfold, fit_params={'callbacks': [es]})
    return scores.mean()

# Run Bayesian Optimization
bayes_opt = BayesianOptimization(f=bayesian_optimization_fn, pbounds=params, random_state=42, verbose=2)
bayes_opt.maximize(init_points=5, n_iter=10)

# Get best parameters
best_params = bayes_opt.max['params']
print("Best parameters: ", best_params)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise a

| 1         | 0.6421    | 2.622     | 122.5     | 0.732     | 0.2993    | 16.24     | 1.312     | 1.232     | 4.465     | 0.06015   | 73.73     | 0.02058   | 6.789     |


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise a

| 2         | 0.6272    | 5.827     | 39.78     | 0.1818    | 0.0917    | 22.17     | 2.05      | 2.728     | 2.165     | 0.06122   | 22.55     | 0.2921    | 2.565     |


...

| 13        | 0.6421    | 6.896     | 99.45     | 0.1436    | 0.4031    | 46.92     | 1.7       | 4.285     | 4.662     | 0.04681   | 65.07     | 0.5407    | 5.915     |


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise a

| 14        | 0.7745    | 4.22      | 39.88     | 0.09083   | 0.2001    | 28.24     | 2.854     | 4.027     | 1.792     | 0.03756   | 87.14     | 0.9164    | 4.225     |


C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ashle\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise a

| 15        | 0.7439    | 1.782     | 125.2     | 0.3827    | 0.4947    | 33.8      | 2.34      | 4.95      | 3.439     | 0.056     | 99.72     | 0.8315    | 0.2311    |
Best parameters:  {'activation': 3.0287604249400606, 'batch_size': 74.9573845650898, 'dropout': 0.611553994605082, 'dropout_rate': 0.1630441803457945, 'epochs': 48.894711988229375, 'kernel': 1.345756028305868, 'layers1': 3.9507756672452916, 'layers2': 2.278843001320099, 'learning_rate': 0.014025416057398328, 'neurons': 50.40999819079428, 'normalization': 0.7535847366029452, 'optimizer': 1.062690767527204}


## 5. CNN model with optimized hyperparameters

### 5.1 Run the CNN model

In [50]:
# Optimized parameters
epochs = int(round(48.894711988229375))  # ~49
batch_size = int(round(74.9573845650898))  # ~75
n_hidden = int(round(50.40999819079428))  # ~50
dropout_rate = 0.1630441803457945
kernel_size = int(round(1.345756028305868))  # ~1
activation = 'softsign'  # Mapped from 3
optimizer = 'Adam'  # Mapped from 1
learning_rate = 0.014025416057398328  # From optimization
layers1 = int(round(3.9507756672452916))  # Approx 4
layers2 = int(round(2.278843001320099))  # Approx 2

# Input dimensions
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

# Set up the model
model = Sequential()

# Add Conv1D layer
model.add(Conv1D(n_hidden, kernel_size=kernel_size, activation=activation, input_shape=(timesteps, input_dim)))

# Add a fully connected Dense layer (layers1 neurons)
model.add(Dense(layers1, activation=activation))

# Add another Dense layer (layers2 neurons)
model.add(Dense(layers2, activation=activation))

# Add a Dropout layer
model.add(Dropout(dropout_rate))

# Add a MaxPooling layer
model.add(MaxPooling1D())

# Flatten the output for dense layers
model.add(Flatten())

# Final layer with n_classes neurons, sigmoid activation for binary classification per station
model.add(Dense(n_classes, activation='sigmoid'))

# Compile the model with the new learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    verbose=2
)

Epoch 1/49


C:\Users\ashle\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


215/215 - 2s - 7ms/step - accuracy: 0.0890 - loss: 0.3141 - val_accuracy: 0.0980 - val_loss: 0.2639
Epoch 2/49
215/215 - 0s - 1ms/step - accuracy: 0.1195 - loss: 0.2592 - val_accuracy: 0.1104 - val_loss: 0.2484
Epoch 3/49
215/215 - 0s - 1ms/step - accuracy: 0.1219 - loss: 0.2492 - val_accuracy: 0.0977 - val_loss: 0.2393
Epoch 4/49
215/215 - 0s - 1ms/step - accuracy: 0.1260 - loss: 0.2425 - val_accuracy: 0.1068 - val_loss: 0.2354
Epoch 5/49
215/215 - 0s - 1ms/step - accuracy: 0.1261 - loss: 0.2396 - val_accuracy: 0.1073 - val_loss: 0.2246
Epoch 6/49
215/215 - 0s - 1ms/step - accuracy: 0.1247 - loss: 0.2350 - val_accuracy: 0.0956 - val_loss: 0.2296
Epoch 7/49
215/215 - 0s - 1ms/step - accuracy: 0.1281 - loss: 0.2316 - val_accuracy: 0.1028 - val_loss: 0.2165
Epoch 8/49
215/215 - 0s - 1ms/step - accuracy: 0.1312 - loss: 0.2270 - val_accuracy: 0.1120 - val_loss: 0.2327
Epoch 9/49
215/215 - 0s - 1ms/step - accuracy: 0.1251 - loss: 0.2268 - val_accuracy: 0.1204 - val_loss: 0.2250
Epoch 10/49


### 5.2 Make predictions

In [52]:
# Make predictions on the test set
predictions = model.predict(X_test)

# Print raw predictions (probabilities for each station)
print("Raw predictions:")
print(predictions)

# Threshold predictions for binary classification (set threshold at 0.5)
threshold = 0.5
binary_predictions = (predictions > threshold).astype(int)

# Print the binary predictions (0s and 1s for each station)
print("Binary predictions:")
print(binary_predictions)

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step
Raw predictions:
[[6.66572771e-04 1.01177962e-02 3.61590344e-03 ... 3.78548776e-07
  1.81845360e-04 6.06045965e-03]
 [6.66505075e-04 1.07485674e-01 6.82862289e-03 ... 3.23179762e-07
  9.78834578e-06 1.41085626e-03]
 [9.80345786e-01 9.90497410e-01 5.77144504e-01 ... 3.63389859e-07
  7.12511409e-03 2.16939058e-02]
 ...
 [2.42146119e-01 4.04266715e-01 2.84969568e-01 ... 8.19532215e-06
  4.11906727e-02 9.22356322e-02]
 [1.69025952e-04 1.65806692e-02 1.97939272e-03 ... 3.90543590e-07
  2.77538425e-06 1.62554882e-03]
 [1.26446728e-04 1.17001766e-02 1.76023331e-03 ... 3.83410054e-07
  8.07872220e-06 1.58024242e-03]]
Binary predictions:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


### 5.3 Classification reports

In [54]:
# Run a classsification report

# Generate predictions on training set
train_predictions = model.predict(X_train)
train_binary_predictions = (train_predictions > threshold).astype(int)

# Print classification reports for training and test sets
print("Training Set Classification Report:")
print(classification_report(y_train, train_binary_predictions))

print("Test Set Classification Report:")
print(classification_report(y_test, binary_predictions))

503/503 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step
Training Set Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.80      0.83      3984
           1       0.81      0.79      0.80      5625
           2       0.85      0.86      0.85      5237
           3       0.86      0.88      0.87      3114
           4       0.86      0.84      0.85      3480
           5       0.92      0.67      0.78      3486
           6       0.84      0.78      0.81      2696
           7       0.79      0.81      0.80      4463
           8       0.89      0.90      0.89      3370
           9       0.81      0.94      0.87      7200
          10       0.84      0.78      0.81      3381
          11       0.82      0.72      0.77      2533
          12       0.00      0.00      0.00         0
          13       0.92      0.96      0.94      2757
          14       0.00      0.00      0.00       837

   micro avg       0.84      0.82      0.83     52163
 

C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\ashle\anaconda3\Lib\site-package

### 5.4 Confusion matrices

In [56]:
# Define the labels (1 = pleasant, 0 = unpleasant)
labels = ['unpleasant', 'pleasant']

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([labels[y] for y in Y_true.flatten()])
    Y_pred = pd.Series([labels[y] for y in Y_pred.flatten()])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# Evaluate and print the confusion matrix
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Apply threshold to get binary predictions
print(confusion_matrix(y_test, y_pred))

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step
Pred        pleasant  unpleasant
True                            
pleasant       17875        3978
unpleasant      3410       78012


In [58]:
#Accuracy = (TP + TN) / (TP + TN + FP + FN)

Accuracy = (17875+78012) / (17875+78012+3978+3410)
print('Accuracy: ', Accuracy)

Accuracy:  0.9284628419268942


In [48]:
# Weather stations list
weather_stations = ['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 'HEATHROW', 
                    'KASSEL', 'LJUBLJANA', 'MAASTRICHT', 'MADRID', 'MUNCHENB', 'OSLO', 'SONNBLICK', 
                    'STOCKHOLM', 'VALENTIA']

# Define the labels
labels = ['pleasant', 'unpleasant']

# Loop over each weather station to calculate and print the confusion matrix
for i, station in enumerate(weather_stations):
    # Get the true labels for the current station (y_test[:, i])
    true_labels = y_test[:, i]  # Using the index of the weather station in y_test
    
    # Get the predicted labels for the current station (binary_predictions[:, i])
    predicted_labels = binary_predictions[:, i]
    
    # Create the confusion matrix
    cm = confusion_matrix(true_labels, predicted_labels)
    
    # Convert confusion matrix to DataFrame for better readability
    cm_df = pd.DataFrame(cm, columns=labels, index=labels)
    
    # Print the confusion matrix for this station
    print(f"Confusion Matrix for {station}:")
    print(cm_df)
    print("\n" + "-"*50 + "\n")

Confusion Matrix for BASEL:
            pleasant  unpleasant
pleasant        4972         233
unpleasant        82        1598

--------------------------------------------------

Confusion Matrix for BELGRADE:
            pleasant  unpleasant
pleasant        4359         159
unpleasant       158        2209

--------------------------------------------------

Confusion Matrix for BUDAPEST:
            pleasant  unpleasant
pleasant        4513         178
unpleasant        59        2135

--------------------------------------------------

Confusion Matrix for DEBILT:
            pleasant  unpleasant
pleasant        5385         155
unpleasant       134        1211

--------------------------------------------------

Confusion Matrix for DUSSELDORF:
            pleasant  unpleasant
pleasant        5253         178
unpleasant       112        1342

--------------------------------------------------

Confusion Matrix for HEATHROW:
            pleasant  unpleasant
pleasant        5250    